In [ ]:
import numpy as np
import random
import math

# PK Kryptographie und das Merkle-Hellman knapsack Kryptosystem

* **Author:** Mike Nöthiger
* **Datum:** 29.05.2020
* **Version:** 1.1

Dieses Dokument beschäftigt sich mit der Grundfrage der Public Key (PK) Kryptographie sowie dem Merkle-Hellman Knapsack Kryptosystem. Es werden die mathematischen Grundlagen von Merkle-Hellman Knapsack vertieft sowie deren Umsetzung in Python erarbeit. In Kapitel 7 wird ein Beispiel der Implementierung demonstriert.

**Inhalt:**

* 1. Kapitel: Einführung
* 2. Kapitel: Public Key Kryptographie
* 3. Kapitel: Subset Sum Problem
* 4. Kapitel: Grundidee Merkle-Hellman Knapsack
* 5. Kapitel: Utility Funktionen
* 6. Kapitel: Kryptosystem Spezifikation und Umsetzung
* 7. Kapitel: Beispiel
* 8. Kapitel: Einschränkungen

# Public Key Kryptographie

Die Public Key (PK) Kryptographie beschäftigt sich mit der Frage: Kann man den Ver- und Entschlüsselungsprozess so aufteilen, sodass mit einem Schlüssel ver- und mit einem anderen Schlüssel entschlüsselt wird?

So entsteht eine Asymmetrie in Ver-/Entschlüsseln, die es einem erlaubt, einen Teil des Schlüsselpaares geheim zu halten (Private Key) und den anderen Teil zu veröffentlichen (Public Key). Mit dem Public Key verschlüsselte Nachrichten können dann nur mit dem Private Key entschlüsselt werden, sodass ein Angreifer der auch im Besitz des Public Keys ist nicht die Möglichkeit hat die Nachricht zu entschlüsseln.

Als Analogie wird der Public Key oft mit einem Vorhängeschloss verglichen. Jeder kann das Vorhängeschloss schliessen (=Nachricht mit dem Public Key verschlüsseln) aber nur der Besitzer des Schlüssels kann das Schloss wieder öffnen (=Cipher mit dem Private Key entschlüsseln).

Die PK Kryptographie addressiert unter anderem auch das Schlüsselaustauschproblem der symmetrischen Verschlüsselung. Dort wird ein sicherer Kanal benötigt über welchen der Schlüssel ausgetauscht wird. Das ist vielleicht praktikabel, wenn man einem Freund einen Schlüssel persönlich übergibt. Aber es wird zum Hindernis, wenn man wie in der modernen Computer Kommukation dauernd einzelne Nachrichten mit fremden Computern/Servern sicher und schnell austauschen will. PK Kryptographie bietet mit der asymmetrischen Verschlüsselung eine praktikable Lösung, indem ein Schlüssel veröffentlicht wird der nur für einen Teil der Arbeit benutzt werden kann (Ver- oder Entschlüsselung). Es ist auch gängig, einen symmetrischen Schlüssel mittels PK Kryptographie verschlüsselt auszutauschen und dann über symmetrische Verschlüsselung zu kommunizieren. Mögliche Gründe dafür könnten effizientere Verschlüsselungsverfahren sein oder einen höhere Sicherheitsstandards.

Je nach Kryptosystem, verhalten sich die Keys (Public/Private) komplementär; das heisst der eine kann jeweils die Arbeit des anderen rückgängig machen. So können beide zur ver- und entschlüsselung benutzt werden. Jedoch ist es immer der Fall, dass ein Key nicht seine eigene Arbeit rückgängig machen kann (dies würde der symmetrischen Kryptographie entsprechen.) Kryptosysteme die diese komplementäre Eigenschaft besitzen, haben den Vorteil, dass man sie auch zur elektronischen Signatur benutzen kann: Ein Private Key Besitzer verschlüsselt eine Signatur und jeder im Besitz des Public Keys kann die Signatur durch entschlüsseln verifizieren.

In der Umsetzung von PK Kryptosystemen sucht man sich Probleme, die im Spezialfall effizient lösbar sind, im Allgemeinfall jedoch nicht (effizient im Sinne der Speicher-/Laufzeitkomplexität). Der Private Key wird so konstruiert, dass der Spezialfall eines Problems vorliegt. Danach wird aus dem Private Key ein Public Key berechnet, mit dem Ziel den Spezialfall in einen schwer lösbaren Allgemeinfall zu wandeln. Die Berechnung des Public Keys muss schwer Umkehrbar sein (siehe [Einwegfunktionen](https://en.wikipedia.org/wiki/One-way_function)). Die in der Berechnung benutzten Parameter sind Teil des Private Keys. Sie werden später als Hintertür (Trapdoor) benutzt, um den Allgemeinfall wieder in den Spezialfall zu wandeln, welcher dann effizient gelöst werden kann. Das Trapdoor funktioniert, weil bei der Berechnung des Public Keys invertierbare Information mitgereicht wird, diese Information bleibt erhalten selbst über die Public Key Verschlüsselung hinaus und kann danach mit der Trapdoor Information invertiert werden, sodass der Spezialfall des Problems vorliegt.

Im folgenden eine einfache Veranschaulichung der Weiterreichung von Information:

* Gegeben: $a, b, c \in \mathbb N$, $a|b$ und $a|c$
* Aus der Teilbarkeit folgt: $b = a \cdot x_1$, $c = a \cdot x_2$
* $a$ teilt nun beliebige Linearkombinationen von $b, c$: $yc + zb = yax_1 + zax_2 = a(yx_1 + zx_2)$

Betrachtet man $a$ als geheime Trapdoor Information und $(b, c)$ als öffentliche Information, so hat der Besitzer von $a$ die Gewissheit, dass er $yc + zb$ teilen können wird. Basierend auf diesem Fakt könnte er nun versuchen ein Kryptosystem zu bauen, dass durch diese Teilbarkeit ein schwieriges Problem in ein einfaches Problem wandelt. Umgekehrt ist es für ein Besitzer von $(b, c)$ schwierig zu sagen, welches $a$ benutzt wurde, weil es vermutlich mehrere Zahlen gibt, die $b$ und $c$ teilen. Natürlich ist das ein triviales, nicht praktikables Beispiel es soll jedoch das Prinzip veranschaulichen. Der Trick ist zu wissen, dass die Trapdoor Information im verschlüsselten Text vorhanden sein wird (hier $a$ als Faktor vorhanden in $b$ und $c$) und diese sich dann zu Nutze machen. Im Kapitel 6.3 wird unter anderem gezeigt, wie das Trapdoor bei Merkle-Hellman zum Zuge kommt.

An der Umsetzung von PK Kryptosystemen wird klar, dass:
* Das zugrunde liegende Problem auch zukünftig schwer lösbar bleiben muss, damit das Kryptosystem tauglich bleibt
* Die Berechnung $Private\ Key \implies Public\ Key$ potentieller Angriffspunkt/Schwachstelle ist (gerade bei Merkle-Hellman, wie sich später herausstellen wird)
* Alle Sicherheit im Schlüssel liegt, nämlich dem Private Key mitsamt Trapdoor Information

# Subset Sum Problem

Das Merkle-Hellman Knapsack Kryptosystem basiert auf dem Untersummenproblem (subset sum problem, ssp). Dieses lautet wiefolgt:

**Gegeben** ein Vektor $A := (a_1, ..., a_n), A \in \mathbb N^n$ sowie $s \in \mathbb N$

**Finde** $X = (x_1, ..., x_n) \in \{0, 1\}^n$, sodass $s = \sum_{i=1}^n a_i x_i$

Es ist kein Algorithmus bekannt, der ein $X$ für ein zufälliges $A$ in effizienter Zeit findet. Ein naiver brute force Algorithmus hätte eine worst case Laufzeit von $O(2^n n)$, ein optimierter Algorithmus käme auf $O(2^{n/2})$ (von [Wikipedia](https://en.wikipedia.org/wiki/Subset_sum_problem#Complexity)).

## Spezialfall superwachsender Vektor

Ein superwachsender Vektor $A = (a_1, ..., a_n)$ erfüllt die Bedingung:

$$
a_{k+1} > \sum_{i=1}^{k} a_k
$$

Jedes Element im Vektor ist grösser als die Summe all seiner Vorgänger. 

Ist $A$ ein superwachsenden Vektor, dann ist das ssp effizient in $O(n)$ lösbar. Falls es eine Lösung für das Problem gibt, ist sie eindeutig. Sei $a_n$ eine Komponente von $A$, falls $a_n \leq s$, dann muss $a_n$ zwingend in der Untersumme vorkommen, denn für die Summe der Vorgänger von $a_n$ gilt

$$
\sum_{i = 1}^{n-1} a_i < a_n \leq s
$$ 

Womit die restlichen Komponenten zu klein wären um als Summe $s$ zu bilden. Daraus folgt ein Algorithmus der beginnend bei der grössten Komponente absteigend alle Komponente prüft die in $s$ passen und so das ssp in $O(n)$ löst (siehe `solve_ssp`).

In [ ]:
def solve_ssp(s, B):
    """
    Solve subset sum problem for a superincreasing vector B.
    
    That is, find vector X with len(X) = len(B), X in {0,1}^n such that B*X=s (scalar product)
    
    Time complexity: O(n), n=len(B)
    
    :param s: sum to solve
    :param B: super increasing vector
    :return: list X = (x_1, ..., x_n) with X in {0,1}^n and n=len(B)
    """
    X = []
    for subsum in reversed(B):
        if s >= subsum:
            X.append(1)
            s -= subsum
        else:
            X.append(0)
    if s > 0:
        raise Exception('subset sum problem not solvable, remainder:', s)
    return list(reversed(X))

# Test ssp algorithm, example from lecture (TheorieKap5 Beispiel 5.3)
s = 40
B = [4, 6, 12, 24, 50, 100]
X = solve_ssp(s, B)
np.testing.assert_array_equal(X, [1, 0, 1, 1, 0, 0])
X

# Grundidee Merkle-Hellman knapsack 

Die Idee vom Merkle-Hellman knapsack Kryptosystem ist es, ein superwachsenden Vektor $B$ in einen nicht superwachsenden (möglichst zufälligen) Vektor $A$ zu transformieren. Mit $A$ werden dann schwer lösbare Untersummen erzeugt (der Ciphertext). Es gibt jedoch ein [trapdoor](https://en.wikipedia.org/wiki/Trapdoor_function) bestehend aus 2 teilerfremden Zahlen $w$ und $m$, mit welchen man die $A$-Untersummen in einfach lösbare $B$-Untersummen zurückführen kann (mittels [multiplikativer Inverse](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse)). Das Trapdoor funktioniert, weil $w$ und $m$ auch zur Erzeugung von $A$ benutzt wurden (modulare Multiplikation), womit ein Zusammenhang zwischen $A$ und $B$ herrscht, dieser pflanzt sich quasi fort und kann zu einem späteren Zeitpunkt als Hintertür benutzt werden um $A$-Untersummen in $B$-Untersummen zurück zu wandeln (vgl. **Kryposystem Spezifikation**).

Der gesamte Verschlüsselungsprozess geht wie folgt:

1. Superwachsenden Vektor $B$ erzeugen
2. Aus $B$ mittels modularer Multiplikation $A$ erzeugen, wobei die Länge erhalten wird $len(B) = len(A)$
3. Den Klartext in einen Bitstring codieren und diesen in Blöcke der Grösse $len(A)$ aufteilen
4. Blockweise Untersummes bilden, indem $Block * A$ skalar Multipliziert wird (=Ciphertext)
5. Die $A$-Untersummen mittels multiplikativer Inverse in $B$-Untersummen zurückführen
6. Die $B$-Untersummen mittels ssp lösen, der daraus resultierende Bitstring wieder decodieren (=Klartext)

# Utility Funktionen

Im folgenden einige utility Funktionen, die in der weiteren Implementierung vom Merkle-Hellman Kryptosystem benötigt werden, hier aber nicht weiter diskutiert werden. (Der Fokus liegt auf dem eigentlichen Kryptosystem.)

Die Funktionen sind jedoch im Code gut dokumentiert.

In [ ]:
# =================
# Utility Functions
# =================

def text_to_bitarray(text):
    """
    Convert a `text` into a bitarray, using 8 bits for each character, padding if necessary.
    
    Raises exception if text contains a character that is more than 8 bits in the unicode table.
    
    :pararm text: text as a string
    :return: np array of size len(text)*8
    """
    bitarray = []
    for char in text:
        # Converts the character to a bitstring and the bitstring to a bit array
        #   {0:08b}.format converts an integer into a bitstring of length 8, padding 0 from left if necessary
        #   see https://docs.python.org/3/library/string.html#formatspec
        bitchunk = [int(bit) for bit in '{0:08b}'.format(ord(char))]
        if len(bitchunk) > 8:
            raise Exception('only 8 bit characters from unicode supported')
        bitarray += bitchunk
    return np.array(bitarray)

def bitarray_to_text(bitarray):
    """
    Convert bitarray back to text
    
    len(bitarray) % 8 must be equal to 0 because each character is considered 8 bits in length.
    
    :param bitarray: np array containing bits (integers)
    :return: string representing the decoded bitarray using unicode
    """
    bitchunks = np.split(bitarray, len(bitarray)/8)
    powers_of_two = 2**np.arange(8)[::-1]
    text = ''
    for chunk in bitchunks:
        integer = chunk.dot(powers_of_two)
        if integer == 0:
            continue
        text += chr(integer)
    return text
    
def bitarray_to_blocks(bit_array, block_length):
    """
    Split a bitarray into blocks of size `block_length`. 
    
    The last block is suffixed with zeros such that all blocks are of equal length.
    (Should be consider when decrypting.)
    
    :param bit_array: np array of bits
    :param block_length: length of each block
    :return: array of np arrays, each np array of length `block_length`
    """
    suffix_len = block_length - len(bit_array) % block_length
    if suffix_len == block_length:
        suffix_len = 0
    bit_array = np.concatenate((bit_array, [0] * suffix_len), axis=0)
    return np.split(bit_array, len(bit_array) / block_length)

def extended_euclid(a, b):
    """
    Find greatest common divisor (gcd) of a and b using extended euclid.

    :return: (g,f1,f2) where g=gcd(a,b) and f1*a + f2*b = g
    """
    af = [1, 0]
    bf = [0, 1]
    while (a > 0) and (b > 0):
        if a < b:
            f = b // a
            bf[0] -= f * af[0]
            bf[1] -= f * af[1]
            b -= f*a
        else:
            f = a // b
            af[0] -= f * bf[0]
            af[1] -= f * bf[1]
            a -= f*b
    if a == 0:
        return b, bf[0], bf[1]
    else:
        return a, af[0], af[1]

def multiplicative_inverse(w, m):
    """
    Calculate multiplicative inverse w^-1, such that w*w^-1 mod m = 1
    
    :param w: number to invert
    :param m: modulus
    :return: w^-1
    """
    g, a, b = extended_euclid(w, m)
    if g != 1:
        raise Exception('gcd must be 1 but was %d' % g)
    if a < 0:
        f = math.ceil(abs(a)/m)
        a += f*m
    return a

# Kryptosystem Spezifikation

## Schlüsselerzeugung

1. Superwachsenden Vektor erzeugen

$$
B = (b_1, ..., b_n),\ b_i \in \mathbb N,\ n \in \mathbb N\\
b_{k+1} > \sum_{i=1}^k b_i
$$

2. Wähle zufälliges $m \in \mathbb N$, sodass

$$
m > \sum_{i=1}^n b_i
$$
3. Wähle zufälliges $w \in \mathbb N$, sodass $w$ und $m$ teilerfremd sind

$$
ggT(w, m) = 1
$$

4. Private Key besteht aus

$$
(B, m, w)
$$

5. Berechne den Public Key

$$
A = (a_1, ..., a_n) \\ 
a_i = b_i \cdot w\ mod\ m
$$

**Anmerkungen**
* $m$ muss grösser als die Summe des superwachsenden Vektors sein, ansonsten könnten unterschiedliche Klartexte denselben Ciphertext erzeugen.
* $w$ und $m$ müssen teilerfremd sein, damit ein modulares Inverses $w^{-1}\ mod\ m$ existiert.

In [ ]:
def generate_private_key(n):
    """
    Generate a random private key.
    
    :param n: Length of the private key (this will determine the bitstring length each block being encrypted)
    :return: the private key as a tupel (B, w, m). B is an np array, w and m are scalars.
    """
    B = []
    random_range = 100
    summ = 0
    for i in range(n):
        element = random.randrange(summ+1, summ+random_range)
        B.append(element)
        summ += element
    B = np.array(B)
    m = random.randrange(np.sum(B)+1, 2*(np.sum(B)+1))
    w = random.randrange(m)
    while math.gcd(m, w) != 1:
        w = random.randrange(m)
        
    return B, w, m

def create_public_key(private_key):
    """
    Create public key from a private key.
    
    :param private_key: private key tupel (B, w, m) as returned by generate_private_key()
    :return: public key A which is a np array of the same shape as B
    """
    B, w, m = private_key
    return B * w % m

## Verschlüsselung

Den Klartext codieren und in $n$-bit Blöcke unterteilen, sodass ein Block die Form $X = (x_1, ..., x_n)$, $X \in \{0,1\}^n$ hat. Danach Blockweise verschlüsseln:

$$
y = \sum_{i=1}^n a_i \cdot x_i
$$

$y$ ist eine einzelne Zahl, eine Untersumme von $A$ welche die Verschlüsselung eines Klarttextblocks repräsentiert. Der vollständige Ciphertext bei $k$ Blöcken ist dann $Y = (y_1, ..., y_k)$.

In [ ]:
def encrypt(plaintext, public_key):
    """
    Encrypt `plaintext` using the `public_key`.
    
    I.e. encodes text into binary, splits binary into blocks and encrypts each block by calculating
    the subset sum of public_key determined by the ones in the corresponding block.
    
    :param text: text to encrypt as a string
    :param public_key: public key as returned by create_public_key()
    :return: encrypted text (i.e. cipher) as np array containing the subset sum for each block
    """
    bitarray = text_to_bitarray(plaintext)
    bitblocks = bitarray_to_blocks(bitarray, len(public_key))
    return np.array([block.dot(public_key) for block in bitblocks])

## Entschlüsselung

1. Schwierig lösbare $A$-Untersummen in einfach lösbare $B$-Untersummen überführen

$$
Z = (z_1, ..., z_n) \\ 
z_i = w^{-1} \cdot y_i\ mod\ m
$$

2. Untersummen Problem lösen um wieder codierten Klartext zu erhalten. $X_i$ ist hier ein einzelner Klartextblock.

$$
X_i = solve\_ssp(z_i, B)
$$

3. Die $X_i$-Blöcke zusammenfügen und decodieren um  den Klartext zu erhalten

**Beweis, dass $z$ Untersummenproblem von $B$ ist**

Es mag auf den ersten Blick klar ersichtlich sein, dass bei der Transformation in Schritt 1. tatsächlich das Untersummen Problem vom allgemeinen Vektor $A$ in eine Untersummen Problem vom superwachsenden Vektor $B$ zurückgeführt wird.

Im folgenden werden die bisherigen Definitionen benutzt um durch Substitution zu zeigen, dass $z$ tatsächlich eine Untersumme von $B$ ist. Wir beziehen uns dazu auf einen einzelnen Klartextblock $X=(x_1, ..., x_n)$ sowie einen gegebenen Private Key $(B, m, w)$ und den daraus resultierenden Pulic Key $A = (a_1, ..., a_n)$.

$$ 
a_i = b_i \cdot w\ mod\ m \\
y = \sum_{i=1}^n a_i \cdot x_i \iff y = \sum_{i=1}^n (b_i \cdot w\ mod\ m) \cdot x_i \iff y = w \cdot \sum_{i=1}^n b_i \cdot x_i\ mod\ m\\
z = w^{-1} \cdot y\ mod\ m \iff z = w^{-1} \cdot w \cdot \sum_{i=1}^n b_i \cdot x_i\ mod\ m \iff z = \sum_{i=1}^n b_i \cdot x_i\ mod\ m
$$

Die letzte Equivalenzumformung führt zu $z = \sum_{i=1}^n b_i \cdot x_i\ mod\ m$ was nichts anderes als eine Untersumme von $B$, dem Private Key ist. Dieses Untersummenproblem kann nun effizient in $O(n)$ gelöst werden, weil $B$ ein superwachsender Vektor ist (vgl. `ssp_solve`).

In [ ]:
def decrypt(cipher, private_key):
    B, w, m = private_key
    w_inv = multiplicative_inverse(w, m)
    # transform A-subset sums into B-subset sums
    cipher = cipher * w_inv % m
    
    bitarray = []
    for s in cipher:
        # solve ssp block wise and collect all the bits arising from it (which are the plain text bits)
        bitblock = solve_ssp(s, B)
        bitarray += bitblock
    # we may have to slice off some bits that were added for encryption, where each block had to be of the same lenth.
    # as we know, each character is encoded in 8 bits, so the whole text needs to be a period of 8 bits
    slice_off = len(bitarray) % 8
    if slice_off > 0:
        bitarray = bitarray[:-slice_off]
    return bitarray_to_text(np.array(bitarray))

# Beispiel

In [ ]:
n = 10
private_key = generate_private_key(n)
public_key = create_public_key(private_key)

plaintext = 'Merkle and Hellman were drinking a beer, meanwhile their system was cracked by Adi Shamir'
cipher = encrypt(plaintext, public_key)
decrypted = decrypt(cipher, private_key)

assert decrypted == plaintext

print('Plaintext\n\t', plaintext)
print('Encoded Plaintext\n\t', text_to_bitarray(plaintext))
print('Private Key\n\t', private_key)
print('Public Key\n\t', public_key)
print('Cipher\n\t', cipher)
print('-------------------------')
print('Decrypted\n\t', decrypted)

# Einschränkungen

Ein Nachteil vom Merkle-Hellman Kryptosystem ist, dass die Ver- und Entschlüsselung nur in eine Richtung funktioniert. Es kann nur mit dem Public Key verschlüsselt werden und mit dem Private Key entschlüsselt. Damit eignet sich Merkle-Hellman nicht für digitale Signaturen, wo der Private Key zur Verschlüsselung benutzt wird und dann jeder im Besitz des dazugehörigen Public Keys, die Signatur durch Entschlüsseln auf deren Authentizität prüfen kann.

Des weiteren wurde das Merkle-Hellman Kryptosystem 1982 von Adi Shamir gebrochen ([Paper](https://ieeexplore.ieee.org/document/4568386)). Die Schwachstelle lag im erzeugen des Public Keys $A = (a_1, ..., a_n)$ ausgehend vom Private Key $(B, w, m)$. In seinem Paper schlägt Adi Shamir einen Algorithmus vor, der in Polynomialzeit ein $W$ und $M$ findet, sodass $a_i \cdot W\ mod\ M$ einen superwachsenden Vektor ergibt, womit man ein potentielles ($w^{-1}$, $m$) Paar vom Privat Key hat.